In [1]:
import os
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML

import pymongo
from pymongo import MongoClient
print ('Mongo version', pymongo.__version__)

Mongo version 3.9.0


#####  The fist step when working with PyMongo is to create a MongoClient to the running Mongod instance.
##### The following code will connect on the default host and port, with specific host and port explicitly

In [2]:
client = MongoClient('localhost', 27017)

> we can also use the following code to access the remote mongodb

from sshtunnel import SSHTunnelForwarder
import pymongo

MONGO_HOST = "REMOTE_IP_ADDRESS"
MONGO_DB = "DATABASE_NAME"
MONGO_USER = "LOGIN"
MONGO_PASS = "PASSWORD"

server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username=MONGO_USER,
    ssh_password=MONGO_PASS,
    remote_bind_address=('127.0.0.1', 27017)
)

server.start()

client = pymongo.MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
db = client[MONGO_DB]

server.stop()

##### Getting a database and a collection within the database

In [3]:
db = client.curd

##### create operations:
##### db.collection.insertOne() --> db.collection.insert_one()
##### db.collection.insertMany()--> db.collection.insert_many()

In [4]:
db.inventory.insert_one(
    {"item": "canvas",
     "qty": 100,
     "tags": ["cotton"],
     "size": {"h": 28, "w": 35.5, "uom": "cm"}})

> In MongoDB, each document stored in a collection requires a unique _id field that acts as a primary key. If an inserted document omits the _id field, the MongoDB driver automatically generates an ObjectId for the _id field.

ObjectIds are small, likely unique, fast to generate, and ordered. ObjectId values consist of 12 bytes, where the first four bytes are a timestamp that reflect the ObjectId’s creation. Specifically:

1. a 4-byte value representing the seconds since the Unix epoch,
2. a 5-byte random value, and
3. a 3-byte counter, starting with a random value.

#### Syntax:
db.database_name.insert_one(           <---Collection
{
    "name" : "sue",                    <---field:value
    "age" : 26,                        <---field:va
    "status" : "pending"               <---field:va
}
)

In [5]:
cursor = db.inventory.find({"item": "canvas"})

In [6]:
pd.DataFrame(list(cursor))

,_id,item,qty,size,tags
0,5c06d624519b1f26a4b217fd,canvas,100,"{'h': 28, 'w': 35.5, 'uom': 'cm'}",[cotton]
1,5dc36a2f631edcc8e61b502b,canvas,100,"{'h': 28, 'w': 35.5, 'uom': 'cm'}",[cotton]
2,5dc36a68293fb9c4b025f5db,canvas,100,"{'h': 28, 'w': 35.5, 'uom': 'cm'}",[cotton]
3,5dc36a93dceb78041aba724a,canvas,100,"{'h': 28, 'w': 35.5, 'uom': 'cm'}",[cotton]


In [7]:
db.inventory.insert_many([
    {"item": "journal",
     "qty": 25,
     "tags": ["blank", "red"],
     "size": {"h": 14, "w": 21, "uom": "cm"}},
    {"item": "mat",
     "qty": 85,
     "tags": ["gray"],
     "size": {"h": 27.9, "w": 35.5, "uom": "cm"}},
    {"item": "mousepad",
     "qty": 25,
     "tags": ["gel", "blue"],
     "size": {"h": 19, "w": 22.85, "uom": "cm"}}])

## Query/Read
#### Query Documents
> MongoDB basically use the BSON file (similar to JSON), <field>:<value>
    for the <value> we can have different type:
    <-- 1. Single value
        2. An array
        3. embedded document, where can again have type 1, 2, and 3 value

In [8]:
db.inventory.drop()
db.inventory.insert_many([
    {"item": "journal",
     "qty": 25,
     "size": {"h": 14, "w": 21, "uom": "cm"},
     "status": "A"},
    {"item": "notebook",
     "qty": 50,
     "size": {"h": 8.5, "w": 11, "uom": "in"},
     "status": "A"},
    {"item": "paper",
     "qty": 100,
     "size": {"h": 8.5, "w": 11, "uom": "in"},
     "status": "D"},
    {"item": "planner",
     "qty": 75, "size": {"h": 22.85, "w": 30, "uom": "cm"},
     "status": "D"},
    {"item": "postcard",
     "qty": 45,
     "size": {"h": 10, "w": 15.25, "uom": "cm"},
     "status": "A"}])

### select all documents in a collection
##### the following operation corresponds to the SQL statement: 
> SELECT * FROM inventory

###### Syntax:
db.datbase_name.find(           <--- collection
    {"field":{"$gt" : 18} },    <--- query criteria
    {"name":1, address: 1}      <--- projection
).limit(5) .                    <--- cursor modifier

In [12]:
cursor = db.inventory.find({})

In [13]:
df1 = pd.DataFrame(list(cursor)) # use list to turn the cursor to an array of documents
df1.head() # return fist n rows, by default n=5

,_id,item,qty,size,status
0,5dc36a9cdceb78041aba724e,journal,25,"{'h': 14, 'w': 21, 'uom': 'cm'}",A
1,5dc36a9cdceb78041aba724f,notebook,50,"{'h': 8.5, 'w': 11, 'uom': 'in'}",A
2,5dc36a9cdceb78041aba7250,paper,100,"{'h': 8.5, 'w': 11, 'uom': 'in'}",D
3,5dc36a9cdceb78041aba7251,planner,75,"{'h': 22.85, 'w': 30, 'uom': 'cm'}",D
4,5dc36a9cdceb78041aba7252,postcard,45,"{'h': 10, 'w': 15.25, 'uom': 'cm'}",A


##### To sepecific equality conditions, use  <field> : <value> 
##### expressions in the query filter documents

In [14]:
cursor = db.inventory.find({"status": "D"})
df1 = pd.DataFrame(list(cursor))
df1.head()

,_id,item,qty,size,status
0,5dc36a9cdceb78041aba7250,paper,100,"{'h': 8.5, 'w': 11, 'uom': 'in'}",D
1,5dc36a9cdceb78041aba7251,planner,75,"{'h': 22.85, 'w': 30, 'uom': 'cm'}",D


#### Specify conditions using query operators
#### list several commonly used operators, more operators can be found with examples on
#### https://docs.mongodb.com/manual/reference/operator/query/#query-selectors

In [15]:
comparison_operators = [
    {"name": "$eq",  "meaning" : "Matches values that are equal to a specified value."},
    {"name": "$gt",  "meaning" : "Matches values that are greater then a specified vlue."},
    {"name": "$gte", "meaning" : "Matches values that are greater than or equal to a specified value."},
    {"name": "$in",  "meaning" : "Matches any of the values specified in an array."},
    {"name": "$lt",  "meaning" : "Matches values that are less than a specified value."},
    {"name": "$lte", "meaning" : "Matches values that are less than or equal to a specified value."},
    {"name": "$ne",  "meaning" : "Matches all values that are not equal to a specified value."},
    {"name": "$nin", "meaning" : "Matches none of the values specified in an array."}
]
df = pd.DataFrame(comparison_operators)
df.style.set_properties(subset=['meaning'], **{'width': '500px'})

,meaning,name
0,Matches values that are equal to a specified value.,$eq
1,Matches values that are greater then a specified vlue.,$gt
2,Matches values that are greater than or equal to a specified value.,$gte
3,Matches any of the values specified in an array.,$in
4,Matches values that are less than a specified value.,$lt
5,Matches values that are less than or equal to a specified value.,$lte
6,Matches all values that are not equal to a specified value.,$ne
7,Matches none of the values specified in an array.,$nin


In [16]:
logical_operators =[
    {"name" : "$and", "meaning" : "Joins query clauses with a logical AND returns all documents that match the conditions of both clauses."},
    {"name" : "$not", "meaning" : "Inverts the effect of a query expression and returns documents that do not match the query expression."},
    {"name" : "$nor", "meaning" : "Joins query clauses with a logical NOR returns all documents that fail to match both clauses."},
    {"name" : "$or",  "meaning" : "Joins query clauses with a logical OR returns all documents that match the conditions of either clause."}
]
df = pd.DataFrame(logical_operators)
df.style.set_properties(subset=['meaning'], **{'width': '600px'})

,meaning,name
0,Joins query clauses with a logical AND returns all documents that match the conditions of both clauses.,$and
1,Inverts the effect of a query expression and returns documents that do not match the query expression.,$not
2,Joins query clauses with a logical NOR returns all documents that fail to match both clauses.,$nor
3,Joins query clauses with a logical OR returns all documents that match the conditions of either clause.,$or


In [17]:
array_operators = [
    {"name":"$all","meaning":"Matches arrays that contain all elements specified in the query."},
    {"name":"$elemMatch","meaning":"Selects documents if element in the array field matches all the specified $elemMatch conditions."},
    {"name":"$size","meaning":"Selects documents if the array field is a specified size."}
]
df = pd.DataFrame(array_operators)
df.style.set_properties(subset=['meaning'], **{'width': '600px'})

,meaning,name
0,Matches arrays that contain all elements specified in the query.,$all
1,Selects documents if element in the array field matches all the specified $elemMatch conditions.,$elemMatch
2,Selects documents if the array field is a specified size.,$size


In [18]:
cursor = db.inventory.find({"status": {"$in": ["A", "D"]}})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,item,qty,size,status
0,5dc36a9cdceb78041aba724e,journal,25,"{'h': 14, 'w': 21, 'uom': 'cm'}",A
1,5dc36a9cdceb78041aba724f,notebook,50,"{'h': 8.5, 'w': 11, 'uom': 'in'}",A
2,5dc36a9cdceb78041aba7250,paper,100,"{'h': 8.5, 'w': 11, 'uom': 'in'}",D
3,5dc36a9cdceb78041aba7251,planner,75,"{'h': 22.85, 'w': 30, 'uom': 'cm'}",D
4,5dc36a9cdceb78041aba7252,postcard,45,"{'h': 10, 'w': 15.25, 'uom': 'cm'}",A


#### Specify AND and OR conditions
1. SELECT * FROM inventory WHERE status = "A" AND qty < 30
2. SELECT * FROM inventory WHERE status = "A" OR qty < 30
3. SELECT * FROM inventory WHERE status = "A" AND ( qty < 30 OR item LIKE "p%")

In [19]:
cursor = db.inventory.find({"status": "A", "qty": {"$lt": 30}})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,item,qty,size,status
0,5dc36a9cdceb78041aba724e,journal,25,"{'h': 14, 'w': 21, 'uom': 'cm'}",A


In [20]:
cursor = db.inventory.find(
    {"$or": [{"status": "A"}, {"qty": {"$lt": 30}}]})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,item,qty,size,status
0,5dc36a9cdceb78041aba724e,journal,25,"{'h': 14, 'w': 21, 'uom': 'cm'}",A
1,5dc36a9cdceb78041aba724f,notebook,50,"{'h': 8.5, 'w': 11, 'uom': 'in'}",A
2,5dc36a9cdceb78041aba7252,postcard,45,"{'h': 10, 'w': 15.25, 'uom': 'cm'}",A


In [21]:
cursor = db.inventory.find({
    "status": "A",
    "$or": [{"qty": {"$lt": 30}}, {"item": {"$regex": "^p"}}]})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,item,qty,size,status
0,5dc36a9cdceb78041aba724e,journal,25,"{'h': 14, 'w': 21, 'uom': 'cm'}",A
1,5dc36a9cdceb78041aba7252,postcard,45,"{'h': 10, 'w': 15.25, 'uom': 'cm'}",A


In [22]:
db.inventory.drop()
from bson.son import SON
db.inventory.insert_many([
    {"item": "journal",
     "qty": 25,
     "size": SON([("h", 14), ("w", 21), ("uom", "cm")]),
     "status": "A"},
    {"item": "notebook",
     "qty": 50,
     "size": SON([("h", 8.5), ("w", 11), ("uom", "in")]),
     "status": "A"},
    {"item": "paper",
     "qty": 100,
     "size": SON([("h", 8.5), ("w", 11), ("uom", "in")]),
     "status": "D"},
    {"item": "planner",
     "qty": 75,
     "size": SON([("h", 22.85), ("w", 30), ("uom", "cm")]),
     "status": "D"},
    {"item": "postcard",
     "qty": 45,
     "size": SON([("h", 10), ("w", 15.25), ("uom", "cm")]),
     "status": "A"}])

##### Match an Embedded/Nested Document
1. To specify an equality condition on a field that is an embedded/nested document, use the query filter document { <field>: <value> } where <value> is the document to match.
For example, the following query selects all documents where the field size equals the document { h: 14, w: 21, uom: "cm" }:

In [23]:
cursor = db.inventory.find(
    {"size": SON([("h", 14), ("w", 21), ("uom", "cm")])})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,item,qty,size,status
0,5dc36ac0dceb78041aba7253,journal,25,"{'h': 14, 'w': 21, 'uom': 'cm'}",A


In [24]:
cursor = db.inventory.find(
    {"size": SON([("w", 21), ("h", 14), ("uom", "cm")])})

##### Query on Nested Field
1. To specify a query condition on fields in an embedded/nested document, use dot notation ("field.nestedField").

In [25]:
cursor = db.inventory.find({"size.uom": "in"})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,item,qty,size,status
0,5dc36ac0dceb78041aba7254,notebook,50,"{'h': 8.5, 'w': 11, 'uom': 'in'}",A
1,5dc36ac0dceb78041aba7255,paper,100,"{'h': 8.5, 'w': 11, 'uom': 'in'}",D


In [26]:
cursor = db.inventory.find({"size.h": {"$lt": 15}})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,item,qty,size,status
0,5dc36ac0dceb78041aba7253,journal,25,"{'h': 14, 'w': 21, 'uom': 'cm'}",A
1,5dc36ac0dceb78041aba7254,notebook,50,"{'h': 8.5, 'w': 11, 'uom': 'in'}",A
2,5dc36ac0dceb78041aba7255,paper,100,"{'h': 8.5, 'w': 11, 'uom': 'in'}",D
3,5dc36ac0dceb78041aba7257,postcard,45,"{'h': 10, 'w': 15.25, 'uom': 'cm'}",A


In [27]:
cursor = db.inventory.find(
    {"size.h": {"$lt": 15}, "size.uom": "in", "status": "D"})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,item,qty,size,status
0,5dc36ac0dceb78041aba7255,paper,100,"{'h': 8.5, 'w': 11, 'uom': 'in'}",D


#### Query an Array

In [28]:
db.inventory.drop()
db.inventory.insert_many([
    {"item": "journal",
     "qty": 25,
     "tags": ["blank", "red"],
     "dim_cm": [14, 21]},
    {"item": "notebook",
     "qty": 50,
     "tags": ["red", "blank"],
     "dim_cm": [14, 21]},
    {"item": "paper",
     "qty": 100,
     "tags": ["red", "blank", "plain"],
     "dim_cm": [14, 21]},
    {"item": "planner",
     "qty": 75,
     "tags": ["blank", "red"],
     "dim_cm": [22.85, 30]},
    {"item": "postcard",
     "qty": 45,
     "tags": ["blue"],
     "dim_cm": [10, 15.25]}])

##### Match an Array
1. The following example queries for all documents where the field tags value is an array with exactly two elements, "red" and "blank", in the specified order:

In [29]:
cursor = db.inventory.find({"tags": ["red", "blank"]})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,dim_cm,item,qty,tags
0,5dc36ac6dceb78041aba7259,"[14, 21]",notebook,50,"[red, blank]"


##### 
1. If, instead, you wish to find an array that contains both the elements "red" and "blank", without regard to order or other elements in the array, use the $all operator:

In [30]:
cursor = db.inventory.find({"tags": {"$all": ["red", "blank"]}})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,dim_cm,item,qty,tags
0,5dc36ac6dceb78041aba7258,"[14, 21]",journal,25,"[blank, red]"
1,5dc36ac6dceb78041aba7259,"[14, 21]",notebook,50,"[red, blank]"
2,5dc36ac6dceb78041aba725a,"[14, 21]",paper,100,"[red, blank, plain]"
3,5dc36ac6dceb78041aba725b,"[22.85, 30]",planner,75,"[blank, red]"


##### Query an Array for an Element
1. To query if the array field contains at least one element with the specified value, use the filter { <field>: <value> } where <value> is the element value.

In [31]:
cursor = db.inventory.find({"tags": "red"})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,dim_cm,item,qty,tags
0,5dc36ac6dceb78041aba7258,"[14, 21]",journal,25,"[blank, red]"
1,5dc36ac6dceb78041aba7259,"[14, 21]",notebook,50,"[red, blank]"
2,5dc36ac6dceb78041aba725a,"[14, 21]",paper,100,"[red, blank, plain]"
3,5dc36ac6dceb78041aba725b,"[22.85, 30]",planner,75,"[blank, red]"


In [32]:
cursor = db.inventory.find({"dim_cm": {"$gt": 25}})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,dim_cm,item,qty,tags
0,5dc36ac6dceb78041aba725b,"[22.85, 30]",planner,75,"[blank, red]"


##### Specify Multiple Conditions for Array Elements

In [33]:
cursor = db.inventory.find({"dim_cm": {"$gt": 15, "$lt": 20}})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,dim_cm,item,qty,tags
0,5dc36ac6dceb78041aba7258,"[14, 21]",journal,25,"[blank, red]"
1,5dc36ac6dceb78041aba7259,"[14, 21]",notebook,50,"[red, blank]"
2,5dc36ac6dceb78041aba725a,"[14, 21]",paper,100,"[red, blank, plain]"
3,5dc36ac6dceb78041aba725c,"[10, 15.25]",postcard,45,[blue]


Use $elemMatch operator to specify multiple criteria on the elements of an array such that at least one array element satisfies all the specified criteria.

In [34]:
cursor = db.inventory.find(
    {"dim_cm": {"$elemMatch": {"$gt": 22, "$lt": 30}}})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,dim_cm,item,qty,tags
0,5dc36ac6dceb78041aba725b,"[22.85, 30]",planner,75,"[blank, red]"


In [35]:
cursor = db.inventory.find({"dim_cm.1": {"$gt": 25}})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,dim_cm,item,qty,tags
0,5dc36ac6dceb78041aba725b,"[22.85, 30]",planner,75,"[blank, red]"


In [36]:
cursor = db.inventory.find({"tags": {"$size": 3}})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

,_id,dim_cm,item,qty,tags
0,5dc36ac6dceb78041aba725a,"[14, 21]",paper,100,"[red, blank, plain]"


#### Query an Array of Embedded Documents

In [37]:
db.inventory.drop()
db.inventory.insert_many([
    {"item": "journal",
     "instock": [
         SON([("warehouse", "A"), ("qty", 5)]),
         SON([("warehouse", "C"), ("qty", 15)])]},
    {"item": "notebook",
     "instock": [
         SON([("warehouse", "C"), ("qty", 5)])]},
    {"item": "paper",
     "instock": [
         SON([("warehouse", "A"), ("qty", 60)]),
         SON([("warehouse", "B"), ("qty", 15)])]},
    {"item": "planner",
     "instock": [
         SON([("warehouse", "A"), ("qty", 40)]),
         SON([("warehouse", "B"), ("qty", 5)])]},
    {"item": "postcard",
     "instock": [
         SON([("warehouse", "B"), ("qty", 15)]),
         SON([("warehouse", "C"), ("qty", 35)])]}])

##### Query for a Document Nested in an Array
1. Equality matches on the whole embedded/nested document require an exact match of the specified document, including the field order.

In [38]:
cursor = db.inventory.find(
    {"instock": SON([("warehouse", "A"), ("qty", 5)])})
df1 = pd.DataFrame(list(cursor))
df1.style.set_properties(subset=['instock'], **{'width': '400px'})

,_id,instock,item
0,5dc36acedceb78041aba725d,"[{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]",journal


In [39]:
cursor = db.inventory.find(
    {"instock": SON([("qty", 5), ("warehouse", "A")])})
df1 = pd.DataFrame(list(cursor))
df1.head(10)

""


##### Specify a Query Condition on a Field in an Array of Documents

In [40]:
cursor = db.inventory.find({'instock.qty': {"$lte": 20}})
df1 = pd.DataFrame(list(cursor))
df1.style.set_properties(subset=['instock'], **{'width': '400px'})

,_id,instock,item
0,5dc36acedceb78041aba725d,"[{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]",journal
1,5dc36acedceb78041aba725e,"[{'warehouse': 'C', 'qty': 5}]",notebook
2,5dc36acedceb78041aba725f,"[{'warehouse': 'A', 'qty': 60}, {'warehouse': 'B', 'qty': 15}]",paper
3,5dc36acedceb78041aba7260,"[{'warehouse': 'A', 'qty': 40}, {'warehouse': 'B', 'qty': 5}]",planner
4,5dc36acedceb78041aba7261,"[{'warehouse': 'B', 'qty': 15}, {'warehouse': 'C', 'qty': 35}]",postcard


In [41]:
cursor = db.inventory.find({'instock.0.qty': {"$lte": 20}})
df1 = pd.DataFrame(list(cursor))
df1.style.set_properties(subset=['instock'], **{'width': '400px'})

,_id,instock,item
0,5dc36acedceb78041aba725d,"[{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]",journal
1,5dc36acedceb78041aba725e,"[{'warehouse': 'C', 'qty': 5}]",notebook
2,5dc36acedceb78041aba7261,"[{'warehouse': 'B', 'qty': 15}, {'warehouse': 'C', 'qty': 35}]",postcard


> The following example queries for documents where the instock array has at least one embedded document that contains both the field qty equal to 5 and the field warehouse equal to A:

In [42]:
cursor = db.inventory.find(
    {"instock": {"$elemMatch": {"qty": 5, "warehouse": "A"}}})
df1 = pd.DataFrame(list(cursor))
df1.style.set_properties(subset=['instock'], **{'width': '400px'})

,_id,instock,item
0,5dc36acedceb78041aba725d,"[{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]",journal


> The following example queries for documents where the instock array has at least one embedded document that contains the field qty that is greater than 10 and less than or equal to 20:

In [43]:
cursor = db.inventory.find(
    {"instock": {"$elemMatch": {"qty": {"$gt": 10, "$lte": 20}}}})
df1 = pd.DataFrame(list(cursor))
df1.style.set_properties(subset=['instock'], **{'width': '400px'})

,_id,instock,item
0,5dc36acedceb78041aba725d,"[{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]",journal
1,5dc36acedceb78041aba725f,"[{'warehouse': 'A', 'qty': 60}, {'warehouse': 'B', 'qty': 15}]",paper
2,5dc36acedceb78041aba7261,"[{'warehouse': 'B', 'qty': 15}, {'warehouse': 'C', 'qty': 35}]",postcard


> Combination of Elements Satisfies the Criteria
Find documents where any document nested in the instock array has the qty field greater than 10 and any document (but not necessarily the same embedded document) in the array has the qty field less than or equal to 20:

In [44]:
cursor = db.inventory.find({"instock.qty": {"$gt": 10, "$lte": 20}})
df1 = pd.DataFrame(list(cursor))
df1.style.set_properties(subset=['instock'], **{'width': '400px'})

,_id,instock,item
0,5dc36acedceb78041aba725d,"[{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]",journal
1,5dc36acedceb78041aba725f,"[{'warehouse': 'A', 'qty': 60}, {'warehouse': 'B', 'qty': 15}]",paper
2,5dc36acedceb78041aba7260,"[{'warehouse': 'A', 'qty': 40}, {'warehouse': 'B', 'qty': 5}]",planner
3,5dc36acedceb78041aba7261,"[{'warehouse': 'B', 'qty': 15}, {'warehouse': 'C', 'qty': 35}]",postcard


In [45]:
cursor = db.inventory.find(
    {"instock.qty": 5, "instock.warehouse": "A"})
df1 = pd.DataFrame(list(cursor))
df1.style.set_properties(subset=['instock'], **{'width': '400px'})

,_id,instock,item
0,5dc36acedceb78041aba725d,"[{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]",journal
1,5dc36acedceb78041aba7260,"[{'warehouse': 'A', 'qty': 40}, {'warehouse': 'B', 'qty': 5}]",planner


In [46]:
cursor = db.inventory.find(
    {"instock": {"warehouse":"A","qty":5}})
df1 = pd.DataFrame(list(cursor))
df1.style.set_properties(subset=['instock'], **{'width': '400px'})

,_id,instock,item
0,5dc36acedceb78041aba725d,"[{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]",journal


### Update Operators

db.collection.updateOne()  --> db.collection.update_one()
db.collection.updateMany() --> db.collection.update_many()
db.collection.replaceOne() --> db.collection.replace_one()

#### Syntax:
db.database_name.update_many(         <--- collection
    {"age":{"$lt" : 18} },            <--- update filter
    {"$set":{"status":"reject"} }     <--- update action

)

In [47]:
db.inventory.drop()
db.inventory.insert_many([
    {"item": "canvas",
     "qty": 100,
     "size": {"h": 28, "w": 35.5, "uom": "cm"},
     "status": "A"},
    {"item": "journal",
     "qty": 25,
     "size": {"h": 14, "w": 21, "uom": "cm"},
     "status": "A"},
    {"item": "mat",
     "qty": 85,
     "size": {"h": 27.9, "w": 35.5, "uom": "cm"},
     "status": "A"},
    {"item": "mousepad",
     "qty": 25,
     "size": {"h": 19, "w": 22.85, "uom": "cm"},
     "status": "P"},
    {"item": "notebook",
     "qty": 50,
     "size": {"h": 8.5, "w": 11, "uom": "in"},
     "status": "P"},
    {"item": "paper",
     "qty": 100,
     "size": {"h": 8.5, "w": 11, "uom": "in"},
     "status": "D"},
    {"item": "planner",
     "qty": 75,
     "size": {"h": 22.85, "w": 30, "uom": "cm"},
     "status": "D"},
    {"item": "postcard",
     "qty": 45,
     "size": {"h": 10, "w": 15.25, "uom": "cm"},
     "status": "A"},
    {"item": "sketchbook",
     "qty": 80,
     "size": {"h": 14, "w": 21, "uom": "cm"},
     "status": "A"},
    {"item": "sketch pad",
     "qty": 95,
     "size": {"h": 22.85, "w": 30.5, "uom": "cm"},
     "status": "A"}])

#### Update Documents in a Collection

In [48]:
db.inventory.update_one(
    {"item": "paper"},
    {"$set": {"size.uom": "cm", "status": "P"},
     "$currentDate": {"lastModified": True}})

In [49]:
cursor = db.inventory.find({"item":"paper"})
df1 = pd.DataFrame(list(cursor))
df1
#df1.style.set_properties(subset=['instock'], **{'width': '400px'})

,_id,item,lastModified,qty,size,status
0,5dc36adedceb78041aba7267,paper,2019-11-07 00:52:47.640,100,"{'h': 8.5, 'w': 11, 'uom': 'cm'}",P


In [50]:
db.inventory.update_many(
    {"qty": {"$lt": 50}},
    {"$set": {"size.uom": "in", "status": "P"},
     "$currentDate": {"lastModified": True}})

In [51]:
db.inventory.replace_one(
    {"item": "paper"},
    {"item": "paper",
     "instock": [
         {"warehouse": "A", "qty": 60},
         {"warehouse": "B", "qty": 40}]})

### Delete Operations
db.collection.deleteOne()  --> db.collection.delete_one()
db.collection.deleteMany() --> db.collection.delete_many()


#### Syntax:
db.database.delete_many(       <--- collection
    {"status": "reject"}       <--- delete filter
)

In [52]:
db.inventory.delete_many({"status": "A"})

In [53]:
db.inventory.delete_one({"status": "D"})

## Text Search
#### MongoDB provides text indexes to support text search queries on string content. text indexes can include any field whose value is a string or an array of string elements.

>>> To perform text search queries, you must have a text index on your collection. A collection can only have one text search index, but that index can cover multiple fields.

In [54]:
db.stores.drop()
db.stores.insert_many(
   [
     {"name": "Java Hut", "description": "Coffee and cakes" },
     {"name": "Burger Buns", "description": "Gourmet hamburgers" },
     {"name": "Coffee Shop", "description": "Just coffee" },
     {"name": "Clothes Clothes Clothes", "description": "Discount clothing" },
     {"name": "Java Shopping", "description": "Indonesian goods" }
   ]
)

In [55]:
db.stores.create_index( [ ("name","text"), ("description", "text") ] )

'name_text_description_text'

>>> 😁
$ $text Operator

Use the '$text' query operator to perform text searches on a collection with a text index.

'$text' will tokenize the search string using whitespace and most punctuation as delimiters, and perform a logical OR of all such tokens in the search string.

> find all stores containing any terms from the list “coffee”, “shop”, and “java”

In [56]:
cursor = db.stores.find( { "$text": { "$search": "java coffee shop" } } )
df1 = pd.DataFrame(list(cursor))
df1
#df1.style.set_properties(subset=['instock'], **{'width': '400px'})

,_id,description,name
0,5dc36ae4dceb78041aba726e,Just coffee,Coffee Shop
1,5dc36ae4dceb78041aba726c,Coffee and cakes,Java Hut
2,5dc36ae4dceb78041aba7270,Indonesian goods,Java Shopping


##### Exact Phrase
> find all documents containing “java” or “coffee shop”
"coffee shop" --> \"coffee shop\"

In [57]:
cursor = db.stores.find( { "$text": { "$search": "java \"coffee shop\"" } } )
df1 = pd.DataFrame(list(cursor))
df1

,_id,description,name
0,5dc36ae4dceb78041aba726e,Just coffee,Coffee Shop


#### Term Exclusion
> find all stores containing “java” or “shop” but not “coffee”

In [58]:
cursor = db.stores.find( { "$text": { "$search": "java shop -coffee" } } )
df1 = pd.DataFrame(list(cursor))
df1

,_id,description,name
0,5dc36ae4dceb78041aba7270,Indonesian goods,Java Shopping


#### Sorting

1. MongoDB will return its results in unsorted order by default. However, text search queries will compute a relevance score for each document that specifies how well a document matches the query.

2. To sort the results in order of relevance score, you must explicitly project the '$meta' textScore field and sort on it:



In [59]:
cursor = db.stores.find(
   { "$text": { "$search": "java coffee shop" } },
   { "score": { "$meta": "textScore" } }
).sort( [( "score", { "$meta": "textScore" } )] )

df1 = pd.DataFrame(list(cursor))
df1

,_id,description,name,score
0,5dc36ae4dceb78041aba726e,Just coffee,Coffee Shop,2.25
1,5dc36ae4dceb78041aba726c,Coffee and cakes,Java Hut,1.50
2,5dc36ae4dceb78041aba7270,Indonesian goods,Java Shopping,1.50
